### <a name="runtime">Runtime parameter definition

In [ ]:
aoi_wkt = 'POLYGON((128.6102 38.1604, 128.6102 38.3037, 128.4164 38.3037, 128.4164 38.1604, 128.6102 38.1604))'

**Input identifiers**

This is the Sentinel-2 stack of products' identifiers

In [ ]:
input_identifiers = ('S2A_MSIL2A_20190403T021651_N0211_R003_T52SDH_20190404T105016',
                     'S2B_MSIL2A_20190408T021609_N0211_R003_T52SDH_20190408T045111')

**Data path**

This path defines where the data is staged in. 

In [ ]:
data_path = '/workspace/data'

**Imports**

In [ ]:
import os
os.environ['PREFIX'] = '/opt/anaconda/envs/env_burned_area/'
import sys
sys.path.append(os.path.join(os.environ['PREFIX'], 'conda-otb/lib/python'))
os.environ['OTB_APPLICATION_PATH'] = os.path.join(os.environ['PREFIX'], 'conda-otb/lib/otb/applications')
os.environ['GDAL_DATA'] =  os.path.join(os.environ['PREFIX'], 'share/gdal')
os.environ['PROJ_LIB'] = os.path.join(os.environ['PREFIX'], 'share/proj')
os.environ['GPT_BIN'] = os.path.join(os.environ['PREFIX'], 'snap/bin/gpt')

import otbApplication

import gdal

from helpers import *
from shapely.wkt import loads
from shapely.geometry import box, shape, mapping
from shapely.errors import ReadingError
import shutil
from datetime import datetime
import xml.etree.ElementTree as ET

os.environ['_JAVA_OPTIONS'] = '-Xms1g -Xmx1g'

gdal.UseExceptions()

print("Success!")

### <a name="workflow">Workflow

Set the metadata of the input products.

In [ ]:
results = [
    {
        'cc': '4.844317',
        'startdate': '2019-04-03T02:16:51.0240000Z',
        'enddate': '2019-04-03T02:16:51.0240000Z',
        'identifier': 'S2A_MSIL2A_20190403T021651_N0211_R003_T52SDH_20190404T105016',
        'orbitDirection': 'DESCENDING',
        'wkt': 'POLYGON((127.862827804024 37.8539506483739,129.110953580642 37.8593896852583,129.112474038138 38.8489443331943,127.847248649247 38.8433100584832,127.862827804024 37.8539506483739))'
    },
    {
        'cc': '14.461197',
        'enddate': '2019-04-08T02:16:09.0240000Z',
        'identifier': 'S2B_MSIL2A_20190408T021609_N0211_R003_T52SDH_20190408T045111',
        'orbitDirection': 'DESCENDING',
        'startdate': '2019-04-08T02:16:09.0240000Z',
        'wkt': 'POLYGON((127.862827804024 37.8539506483739,129.110953580642 37.8593896852583,129.112474038138 38.8489443331943,127.847248649247 38.8433100584832,127.862827804024 37.8539506483739))'
    }
]

In [ ]:
for row in results:
    row['wkt'] = loads(row['wkt'])
results

In [ ]:
aoi = loads(aoi_wkt)
(min_lon, min_lat, max_lon, max_lat) = aoi.bounds

In [ ]:
print(aoi)
print (isinstance(aoi, str))
for row in results:
    #print("{0}: {1}".format(row['wkt'], row['wkt'].area))
    ext = analyse(row, aoi, data_path)
    #print(ext)
    
for row in results:
    print(row)


#sentinel2_search = sentinel2_search.merge(sentinel2_search.apply(lambda row: analyse(row, aoi, data_path), axis=1), 
#                                          left_index=True,
#                                          right_index=True)

In [ ]:
os.environ['GDAL_DATA'] = '/opt/anaconda/envs/env_ewf_burned_area/share/gdal/'

In [ ]:
composites = []

bands = ['B12', 'B11', 'B8A']

for row in results:
    vrt_bands = []
    
    for j, band in enumerate(bands):
        
        vrt_bands.append(get_band_path(row, band))
    
    vrt = '{0}.vrt'.format(row['identifier'])
    ds = gdal.BuildVRT(vrt,
                       vrt_bands,
                       srcNodata=0,
                       xRes=10, 
                       yRes=10,
                       separate=True)
    ds.FlushCache()
    
    tif =  '{0}.tif'.format(row['identifier'])
    
    if aoi is None:
        gdal.Translate(tif,
                       vrt,
                       outputType=gdal.GDT_Byte, 
                       scaleParams=[[0, 10000, 0, 255]])
    else:
        gdal.Translate(tif,
                       vrt,
                       projWin=[min_lon, max_lat, max_lon, min_lat],
                       projWinSRS='EPSG:4326',
                       outputType=gdal.GDT_Byte, 
                       scaleParams=[[0, 10000, 0, 255]])
        
        
        
    tif_e =  '{0}_NIR_SWIR_COMPOSITE.tif'.format(row['identifier'])
    
    try:
        contrast_enhancement(tif, tif_e)

        composites.append(tif_e)
    except Exception as e:
        print(str(e))
    
    # os.remove(tif)
    # os.remove(vrt)

### Hot spot

In [ ]:
bands = ['B8A', 'B12']

composites = []

for row in results:
        
    vrt_bands = []
    
    for j, band in enumerate(bands):
        
        vrt_bands.append(get_band_path(row, band))
    
    vrt = '{0}.vrt'.format(row['identifier'])
    ds = gdal.BuildVRT(vrt,
                       vrt_bands,
                       srcNodata=0,
                       xRes=20, 
                       yRes=20,
                       separate=True)
    ds.FlushCache()
    
    tif =  '{0}.tif'.format(row['identifier'])
    
    if aoi is not None:
        gdal.Translate(tif,
                       vrt,
                       projWin=[min_lon, max_lat, max_lon, min_lat],
                       projWinSRS='EPSG:4326',
                       outputType=gdal.GDT_Float32)
    else:
        
        gdal.Translate(tif,
                       vrt,
                       outputType=gdal.GDT_Float32)
        
    
    hot_spot(get_band_path(row, 'B8A'),
             get_band_path(row, 'B12'),
             '{0}_HOT_SPOT.tif'.format(row['identifier']))
    
    # os.remove(tif)
    # os.remove(vrt)

In [ ]:
for row in results:
    
    hot_spot(get_band_path(row, 'B8A'),
             get_band_path(row, 'B12'),
             '{0}_HOT_SPOT.tif'.format(row['identifier']))

In [ ]:
pre_processed = []

resample = dict()
resample['referenceBandName'] = 'B2'

reproject = dict()
reproject['crs'] = 'EPSG:4326'

subset = dict()
subset['geoRegion'] = box(*aoi.bounds).wkt
subset['copyMetadata'] = 'true'

bands = '''<targetBands>
    <targetBand>
      <name>NDWI</name>
      <type>float32</type>
      <expression>(B3 - B8) / (B3 + B8)</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    <targetBand>
      <name>NBR</name>
      <type>float32</type>
      <expression>(B8 - B12) / (B8 + B12)</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    <targetBand>
      <name>valid_pixels</name>
      <type>float32</type>
      <expression>scl_vegetation or scl_not_vegetated ? 1 : 0</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    </targetBands>'''

band_maths = dict()
band_maths['targetBandDescriptors'] = bands 



for row in results:
    print(os.path.join(row['local_path'], row['identifier'] + '.SAFE', 'MTD_MSIL2A.xml'))
    
    read = dict()
    read['file'] = os.path.join(row['local_path'], row['identifier'] + '.SAFE', 'MTD_MSIL2A.xml') #, 'manifest.safe')
    #read['formatName'] = 'SENTINEL-2-MSI-MultiRes-UTM52N'
    
    write = dict()
    write['file'] = 'pre_{}'.format(row['identifier'])

    row['pre_proc'] = 'pre_{}'.format(row['identifier'])
    
    
    pre_processed.append('pre_{}'.format(row['identifier']))

    print("*******")
    print("IDENTIFIER = {0}".format(row['identifier']))
    print("READ = {0}".format(read))
    print("WRITE = {0}".format(write))
    print("*******")
    
    
    pre_processing2(Read=read, 
                 Resample=resample, 
                 Reproject=reproject, 
                 Subset=subset,
                 BandMaths=band_maths,
                 Write=write)

In [ ]:
print(results)

In [ ]:
master_path = "{0}.dim".format(min((r['pre_proc'] for r in results)))
slave_path = "{0}.dim".format(max((r['pre_proc'] for r in results)))

print(master_path)
print(slave_path)

In [ ]:
mygraph = GraphProcessor('/opt/anaconda/envs/env_ewf_burned_area/snap/bin/gpt')
operator = 'Read'

node_id = 'Read_M'

source_node_id = ''

parameters = get_operator_default_parameters(operator)
     
parameters['file'] = master_path 
    
mygraph.add_node(node_id, operator, parameters, source_node_id)

operator = 'Read'

node_id = 'Read_S'

source_node_id = ''

parameters = get_operator_default_parameters(operator)
     
parameters['file'] = slave_path   
    
mygraph.add_node(node_id, operator, parameters, source_node_id)

operator = 'Collocate'

parameters = get_operator_default_parameters(operator)

parameters['masterComponentPattern'] = 'PRE_FIRE_${ORIGINAL_NAME}'
parameters['slaveComponentPattern'] = 'POST_FIRE_${ORIGINAL_NAME}'

source_node_id = dict()

source_node_id['master'] = 'Read_M'

source_node_id['slave'] = 'Read_S'


node_id = 'Collocate'

mygraph.add_node(operator, operator,  parameters, source_node_id)

operator = 'Write'

node_id = 'Write'

source_node_id = 'Collocate'



parameters = get_operator_default_parameters(operator)

parameters['file'] = 'collocated'
parameters['formatName'] = 'BEAM-DIMAP'

mygraph.add_node(node_id, operator, parameters, source_node_id)

In [ ]:
run2(mygraph)

In [ ]:
output_name = 'burned_area_{0}_{1}'.format(datetime.strptime(max(r['enddate'] for r in results)[:19] , '%Y-%m-%dT%H:%M:%S').strftime('%Y%m%d_%H%M%S'),
                                          datetime.strptime(max(r['enddate'] for r in results)[:19] , '%Y-%m-%dT%H:%M:%S').strftime('%Y%m%d_%H%M%S'))

In [ ]:
collocated_input = 'collocated.dim'

read = dict()
read['file'] = collocated_input

bands = '''<targetBands>
    <targetBand>
      <name>dNBR</name>
      <type>float32</type>
      <expression>(PRE_FIRE_valid_pixels == 1 and POST_FIRE_valid_pixels == 1 and ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) > 0.27) ? PRE_FIRE_NBR - POST_FIRE_NBR : -999</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    <targetBand>
      <name>RBR</name>
      <type>float32</type>
      <expression>(PRE_FIRE_valid_pixels == 1 and POST_FIRE_valid_pixels == 1 and ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) > 0.27) ? ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) : -999</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    <targetBand>
      <name>valid_pixels</name>
      <type>float32</type>
      <expression>PRE_FIRE_valid_pixels == 1 and POST_FIRE_valid_pixels == 1</expression>
      <description/>
      <unit/>
      <noDataValue>NaN</noDataValue>
    </targetBand>
    </targetBands>'''

band_maths = dict()
band_maths['targetBandDescriptors'] = bands 

write = dict()
write['file'] = output_name
write['formatName'] = 'GeoTIFF'

burned_area(Read=read, 
            BandMaths=band_maths,
            Write=write)

In [ ]:
band_names = ['dNBR',
              'RBR',
             'valid_pixels']

expressions = ['PRE_FIRE_NDWI >= 0.0 ? 0 : ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) > 0.27 ? PRE_FIRE_NBR - POST_FIRE_NBR : 0',
               'PRE_FIRE_NDWI >= 0.0 ? 0 : ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) > 0.27 ? ((PRE_FIRE_NBR - POST_FIRE_NBR) / (PRE_FIRE_NBR + 1.001)) : 0',
              'PRE_FIRE_valid_pixels == 1 and POST_FIRE_valid_pixels == 1']


metadata= dict()


ds_temp = gdal.Open(output_name + '.tif',  gdal.OF_UPDATE)

for band_index in range(ds_temp.RasterCount):

    metadata= dict()
    metadata['BAND_EXPRESSION'] = expressions[band_index]

    src_band = ds_temp.GetRasterBand(band_index+1)
    src_band.SetMetadata(metadata)
    src_band.SetDescription(band_names[band_index])  

ds_temp.FlushCache()

In [ ]:
severity_threshold = 0.270

In [ ]:
severity_palette = {-999: [0, 0, 0, 255],
                    -1: [159, 159, 159, 0], # grey
                    -0.5: [43, 25, 223, 0], # blue
                    -0.251: [139, 221, 231, 0], # cyan
                    -0.101: [97, 169, 45, 255], # unburned, green 
                    0.099: [250, 254, 76, 0], # yellow
                    0.269: [228, 173, 55, 0], # orange
                    0.439: [202, 59, 18, 0],  # red
                    0.659: [85, 15, 112, 0]} # purple

In [ ]:
raster2rgb(output_name + '.tif',
           severity_palette,
           output_name + '.rgb.tif',
           raster_band=1,
           discrete=True)

##### Clean-up

In [ ]:
for dimap in ['collocated'] + [r['pre_proc'] for r in results]:
    print("Delete {0}.dim".format(dimap))
    os.remove(dimap + '.dim')
    print("Delete {0}.data".format(dimap))
    shutil.rmtree(dimap + '.data') 

#### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.